In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paths
import logging
import asyncio
import matplotlib.pyplot as plt
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
from blockhead.gdax import data

In [ ]:
import pathlib
pair='ETH-USD'

In [ ]:
path = pathlib.Path('output', pair, str(60))

In [ ]:
barfile = path / '2018-02-22'
barfile.absolute()

In [ ]:
bars = data.get_bars('ETH-USD', '2018-01-01', '2018-05-03', 60, 'output')

In [ ]:
bars_all = bars
bars = bars['2018-03-23':]

In [ ]:
%matplotlib notebook

In [ ]:
# mins minute bars, 12/26 ema cross
mins = 60 
bars['ema12'] = bars['close'].ewm(span=mins*12).mean()
bars['ema26'] = bars['close'].ewm(span=mins*26).mean()
bars['signal'] = 0
# go long only when 12 crosses 26
bars.loc[bars['ema12'] > bars['ema26'], 'signal'] = 1

In [ ]:
(fig, axes) = plt.subplots(2, 1, sharex=True)
axes[0].plot(bars[['close', 'ema12', 'ema26']])
axes[1].plot(bars['signal'])

In [ ]:
returns = bars['close'] - bars['close'].shift()

In [ ]:
returns.head()

In [ ]:
returns.sum()

In [ ]:
on = returns.loc[bars['signal'] == 1]
on.sum()

In [ ]:
off = returns.loc[bars['signal'] == 0]
off.sum()

In [ ]:
len(bars)

In [ ]:
returns.mean()/returns.std()

In [ ]:
on.mean()/on.std()

In [ ]:
off.mean()/off.std()

In [ ]:
(fig, axis) = plt.subplots(1, 1, sharex=True)
axis.plot(on.cumsum())
axis.plot(off.cumsum())
axis.plot(returns.cumsum())
axis.legend(['on', 'off', 'all'])

In [ ]:
daily = on.resample('1d').sum()
print(daily.mean(), daily.std(), daily.mean()/daily.std())